## Ligação BD:

In [2]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from bs4 import BeautifulSoup
from Bio.SeqFeature import CompoundLocation
import mysql.connector as SQLC
import mysql.connector

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True 
Cursor = DataBase.cursor()

num= input("Escolha o tamanho máximo das seq: ")
Seq_and_length=[]
just_seq=[]
try:
    sql_Seq_length= f"select Gene.ID_genebank, Gene.length from Gene where Gene.length <{num}"
    Cursor.execute(sql_Seq_length)
    for row in Cursor:
        Seq_and_length.append(str(row)) 
    
    sql_Seq= f"select Gene.ID_genebank from Gene where Gene.length <{num}"
    Cursor.execute(sql_Seq)
    for row in Cursor:
        just_seq.append(str(row))
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
print (Seq_and_length)

if just_seq == []:
        print()
        print('nenhuma seq inferior a esse tamanho, pesquise de novo.')

Escolha o tamanho máximo das seq: 50
["('AAADK0043071.1', 21)", "('AAADS0003348.1', 20)", "('AAADW0009410.1', 21)", "('AAADX0043451.1', 21)", "('ABAAM0346030.1', 20)"]


In [3]:
list_seq=[]

choose= input('escolha a primeira seq com que quer trabalhar: ')
inter= int(choose)
extr= ', '.join(just_seq)
h= extr.replace("(",'')
hh= h.replace(",)",'')
ID_al= hh.split(', ')
select= ID_al[inter]
list_seq.append(select)

choose2= input('escolha a segunda seq com que quer trabalhar: ')
inter2= int(choose2)
extr2= ', '.join(just_seq)
h2= extr2.replace("(",'')
hh2= h2.replace(",)",'')
ID_al2= hh2.split(', ')
select2= ID_al2[inter2]
list_seq.append(select2)

limpar_aspas= ', '.join(list_seq)
retirar= limpar_aspas.replace("'","")
otput_final= retirar.split(', ')
print(otput_final)

escolha a primeira seq com que quer trabalhar: 2
escolha a segunda seq com que quer trabalhar: 3
['AAADW0009410.1', 'AAADX0043451.1']


In [4]:
#SEQ_algoritmos
seq_for_al=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist1= otput_final[0]
idlist2= otput_final[1]
handle1 = Entrez.efetch(db=database, id=idlist1, rettype="gb") 
records1 = SeqIO.read(handle1,"gb")
handle1.close()
s1_=records1.seq
handle2 = Entrez.efetch(db=database, id=idlist2, rettype="gb") 
records2 = SeqIO.read(handle2,"gb")
handle2.close()
s2_=records2.seq

print(s1_)
print(s2_)

C:\Users\guilh\anaconda3\lib\site-packages\Bio\Entrez\__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


GGGATGGAAGATGGCGGCCTA
GGTGGCTAGGGCCGCGCTGGT


["('AAADK0043071.1', 21)", "('AAADS0003348.1', 20)", "('AAADW0009410.1', 21)", "('AAADX0043451.1', 21)", "('ABAAM0346030.1', 20)"]
escolha a primeira seq com que quer trabalhar: 4
GTTTGGTGAACTTCTGCCTA


### Matriz:

In [ ]:
import io

class Mat:
    
    def __init__(self, rows, cols):
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    def numRows (self): return len(self.mat)
    def numCols (self): return len(self.mat[0])
    def __str__(self):
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    def __repr__(self):
        return str(self)
    def __getitem__ (self, n):
        return self.mat[n]

### NW:

In [ ]:
def subst(x, y):
    return 3 if x == y else -1 #match de 3 mismatch -1

class NW:
    def __init__(self, s1, s2, g = -4):
        self.s1 = s1 
        self.s2 = s2 
        self.mat = Mat(len(s1) + 1, len(s2) + 1)
        self.tr  = Mat(len(s1) + 1, len(s2) + 1)

        for L in range(len(s1)):
            self.mat[L + 1][0] = g * (L + 1) 
            self.tr[L + 1][0]  = 'C' 
        for C in range(len(s2)):
            self.mat[0][C + 1] = g * (C + 1) 
            self.tr[0][C + 1]  = 'E'

        for L, x1 in enumerate(s1):
            for C, x2 in enumerate(s2):
                possiveis = [
                    self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
                    self.mat[L+1][C    ] + g,               # Esquerda
                    self.mat[L  ][C + 1] + g,               # Cima
                ]
                dirs = "DEC"
                self.mat[L + 1][C + 1] = max(possiveis)
                self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]
                
    def rebuild(self):
        L = len(self.s1)
        C = len(self.s2)
        S1 = ""
        S2 = ""
    
        dirs = {
            'D' : (-1, -1),
            'E' : ( 0, -1),
            'C' : (-1,  0)
        }
        while L > 0 or C > 0:
            
            DL, DC = dirs[self.tr[L][C]]
            if self.tr[L][C] == "D":
                S1 = self.s1[L - 1] + S1 
                S2 = self.s2[C - 1] + S2
            elif self.tr[L][C] == "E":
                S1 = '-' + S1
                S2 = self.s2[C - 1] + S2
            else:
                S1 = self.s1[L - 1] + S1
                S2 = '-' + S2        
    
            L += DL
            C += DC

        return S1, S2 
    def __repr__(self): 
        cols = "-" + self.s2
        lins = "-" + self.s1
        with io.StringIO("") as S:
            print(' ', *cols, sep = '   ', file = S)  
            for L, linha in zip(lins, self.mat):
                print(L, *[f'{x:3d}' for x in linha], file = S)
            print(file = S)

            print(' ', *cols, file = S) 
            for L, linha in zip(lins, self.tr):
                print(L, *linha, file = S)

            return S.getvalue()
        
alin= NW(s1_,s2_, g= -4)
print (alin)
print()
print(*alin.rebuild(), sep="\n")


### SW:

In [ ]:
class SW:
    def __init__(self, s1, s2, gap_penalty, match_score, mismatch_penalty):
        self.s1 = s1 
        self.s2 = s2 
        self.gap_penalty= gap_penalty
        self.match_score= match_score
        self.mismatch_penalty= mismatch_penalty
        self.score = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
        self.tr = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
       

        for L in range(len(s1)): 
            self.tr[L + 1][0]  = 'E' 
        for C in range(len(s2)):
            self.tr[0][C + 1]  = 'C'
                
        for i in range(1, len(self.s1) + 1):
            for j in range(1, len(self.s2) + 1):
                match = self.score[i - 1][j - 1] + (self.match_score if self.s1[i - 1] == self.s2[j - 1] else self.mismatch_penalty)
                delete = self.score[i - 1][j] + self.gap_penalty
                insert = self.score[i][j - 1] + self.gap_penalty
                self.score[i][j] = max(0, match, delete, insert)
                possiveis= (match, delete, insert,0)
                dirs = 'DEC0'
                self.tr[i][j] = dirs[possiveis.index(self.score[i][j])]
                
                
    def rebuild(self):
    
        L = len(self.s1)
        C = len(self.s2)
            
        max_i, max_j = 0, 0
        max_score = 0
        for i in range(1, L + 1):
            for j in range(1, C + 1):
                if self.score[i][j] > max_score:
                    max_i, max_j = i, j
                    max_score = self.score[i][j]
        i, j = max_i, max_j
        alignD1= "" 
        alignD2= ""
        while self.score[i][j] != 0:
            current_score = self.score[i][j]
            diagonal_score = self.score[i - 1][j - 1]
            up_score = self.score[i][j - 1]
            left_score = self.score[i - 1][j]

            if current_score == diagonal_score + (self.match_score if self.s1[i - 1] == self.s2[j - 1] else self.mismatch_penalty):
                alignD1 += self.s1[i - 1]
                alignD2 += self.s2[j - 1]
                i -= 1
                j -= 1
            elif current_score == left_score + self.gap_penalty:
                alignD1 += self.s1[i - 1]
                alignD2 += "-"
                i -= 1
            elif current_score == up_score + self.gap_penalty:
                alignD1 += "-"
                alignD2 += self.s2[j - 1]
                j -= 1
        
                
        return alignD1[::-1], alignD2[::-1] 
    
    
    
    def __repr__(self): 
        cols = "-" + self.s2
        lins = "-" + self.s1
        with io.StringIO("") as S:
            print(' ', *cols, sep = '   ', file = S)  
            for L, linha in zip(lins, self.score):
                print(L, *[f'{x:3d}' for x in linha], file = S)
            print(file = S)

            print(' ', *cols, file = S) 
            for L, linha in zip(lins, self.tr ):
                print(L, *linha, file = S)

            return S.getvalue()


alignSW= SW(s1_,s2_,-4,3,-1)
print(alignSW)
print(*alignSW.rebuild(), sep="\n")

### Multiple align:

In [ ]:
s3='AATCGGTCA'
s4='GGTTGGGAC'

def consensus(s1_, s2_):
    res = ""
    for x, y in zip(s1_, s2_):
        if x == y:
            res += x
        elif x == '-':
            res += y
        else:
            res += x
    #print (res)
    return res

a1, a2 = NW(s1_, s2_, g = -1).rebuild()
print(a1)
print(a2)
print()
m1 = consensus(a1,a2)
print(m1)
print()
m2, a3 = NW(m1, s3, g = -1).rebuild()
print(m2,a3, sep="\n")
print()
m2= consensus(a2,a3)
print(m2)
print()
m3, a4= NW(m2,s4,g=-1).rebuild()
print(m3)
print(a4)
print()